<a href="https://colab.research.google.com/github/semaHbo/job-clustering-salary-prediction/blob/main/feature_engineering_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Yeni Feature Engineering") \
    .getOrCreate()

Mounted at /content/drive


In [ ]:

train_df = spark.read.parquet("/content/drive/MyDrive/datasets/train_df.parquet")
test_df = spark.read.parquet("/content/drive/MyDrive/datasets/test_df.parquet")

df_all = train_df.union(test_df)


In [ ]:
from pyspark.sql.functions import when

df_all = df_all.withColumn(
    "Remote_Tipi",
    when(df_all["Uzaktan_Calisma_Orani"] == 0, "Yerinde")
    .when((df_all["Uzaktan_Calisma_Orani"] > 0) & (df_all["Uzaktan_Calisma_Orani"] < 100), "Hibrit")
    .otherwise("Uzaktan")
)


In [ ]:
kita_map = {
    "US": "Kuzey Amerika", "CA": "Kuzey Amerika", "MX": "Kuzey Amerika",
    "GB": "Avrupa", "DE": "Avrupa", "FR": "Avrupa", "ES": "Avrupa", "PT": "Avrupa", "NL": "Avrupa", "PL": "Avrupa", "IT": "Avrupa", "GR": "Avrupa", "IE": "Avrupa", "BE": "Avrupa", "CH": "Avrupa",
    "IN": "Asya", "CN": "Asya", "JP": "Asya", "VN": "Asya", "SG": "Asya", "PH": "Asya", "MY": "Asya",
    "BR": "Güney Amerika", "AR": "Güney Amerika", "CO": "Güney Amerika", "CL": "Güney Amerika",
    "AU": "Okyanusya", "NZ": "Okyanusya",
    "ZA": "Afrika", "NG": "Afrika", "KE": "Afrika", "EG": "Afrika",
    "HN": "Orta Amerika",
    "HU": "Avrupa", "RO": "Avrupa", "HR": "Avrupa", "TR": "Avrupa",
}


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Kıta eşleme fonksiyonu
def ulke_to_kita(ulke_kodu):
    return kita_map.get(ulke_kodu, "Bilinmeyen")

# UDF tanımla
ulke_to_kita_udf = udf(ulke_to_kita, StringType())

# Yeni sütunu ekle
df_all = df_all.withColumn("Kita", ulke_to_kita_udf(df_all["Sirket_Ulke"]))


In [ ]:
df_all = df_all.withColumnRenamed("work_year", "Yil")

In [ ]:
df_all.select("Yil").distinct().orderBy("Yil").show()

+----+
| Yil|
+----+
|2020|
|2021|
|2022|
+----+



In [ ]:
df_all.write.mode("overwrite").parquet("/content/drive/MyDrive/datasets/df_enriched.parquet")
